In [8]:
import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import requests
from selenium.common.exceptions import NoSuchElementException ,StaleElementReferenceException

In [15]:
url = 'https://www.cotodigital3.com.ar'

#this func takes the first url all with Beatiful Soup and the names, categories
def take_url():
    #taking the url
    r = requests.get(url)


    soup =BeautifulSoup(r.text,'html.parser')
    #selecting the tags
    classes=soup.find_all('ul',class_='sub_category')
    
    #looking at the href
    links= [i.find('a')['href'] for i in classes]

    #names of categories
    names=[i.a.text   for i in classes]
    #taking out the spaces and extra symbols
    names_categories = [i.replace(' (+)','').strip() for i in names]
                               
    return links ,names_categories
    


#this variable holds the function of categories and first links. This links have to iterate to another pages
first_step = take_url()

In [16]:
first_step[1]

['Golosinas',
 'Panaderia',
 'Snacks',
 'Cereales',
 'Endulzantes',
 'Aderezos y salsas',
 'Infusiones',
 'Conservas',
 'Encurtidos',
 'Harinas',
 'Mermeladas y dulces',
 'Salsas y puré de tomate',
 'Aceites y condimentos',
 'Alimento de bebés y niños',
 'Arroz y legumbres',
 'Especias',
 'Pasta seca, lista y rellenas',
 'Polvo para postres y reposteria',
 'Sopas, caldos, puré y saborizantes',
 'Rebozador y pan rallado',
 'Leche en polvo',
 'Suplementos dietarios',
 'Productos orgánicos',
 'Huevos de pascua',
 'Bebidas sin alcohol',
 'Bebidas con alcohol',
 'Lácteos',
 'Fiambres',
 'Quesos',
 'Carniceria',
 'Aves',
 'Pastas frescas y tapas',
 'Comidas elaboradas',
 'Frutas y verduras',
 'Pescaderia',
 'Huevos',
 'Helados y postres',
 'Comidas congeladas',
 'Hamburguesas y milanesas',
 'Vegetales congelados',
 'Papas congeladas / fritas',
 'Frutas congeladas',
 'Nuggets, patitas y bocaditos',
 'Pescaderia',
 'Lavado',
 'Papeles',
 'Insecticidas',
 'Calzado',
 'Accesorios de limpieza',
 

In [17]:
first_step[0]

['/sitios/cdigi/browse/catalogo-almac%C3%A9n-golosinas/_/N-1y5dh9i',
 '/sitios/cdigi/browse/catalogo-almac%C3%A9n-panaderia/_/N-s3bf1a',
 '/sitios/cdigi/browse/catalogo-almac%C3%A9n-snacks/_/N-10kzbyj',
 '/sitios/cdigi/browse/catalogo-almac%C3%A9n-cereales/_/N-ukd5id',
 '/sitios/cdigi/browse/catalogo-almac%C3%A9n-endulzantes/_/N-1rtbab6',
 '/sitios/cdigi/browse/catalogo-almac%C3%A9n-aderezos-y-salsas/_/N-rv0frc',
 '/sitios/cdigi/browse/catalogo-almac%C3%A9n-infusiones/_/N-dw58vw',
 '/sitios/cdigi/browse/catalogo-almac%C3%A9n-conservas/_/N-1t4efca',
 '/sitios/cdigi/browse/catalogo-almac%C3%A9n-encurtidos/_/N-12rkdi1',
 '/sitios/cdigi/browse/catalogo-almac%C3%A9n-harinas/_/N-842qrm',
 '/sitios/cdigi/browse/catalogo-almac%C3%A9n-mermeladas-y-dulces/_/N-mj4aa8',
 '/sitios/cdigi/browse/catalogo-almac%C3%A9n-salsas-y-pur%C3%A9-de-tomate/_/N-os1anu',
 '/sitios/cdigi/browse/catalogo-almac%C3%A9n-aceites-y-condimentos/_/N-18r69ct',
 '/sitios/cdigi/browse/catalogo-almac%C3%A9n-alimento-de-beb%C3

In [18]:
def take_product(categorias,url):



    r = requests.get(url)


    soup =BeautifulSoup(r.text,'html.parser')

    products = soup.find_all('li',class_='clearfix')

    #url_product=[]
    #description_product=[]
    #precio_contado=[]
    #promocion=[]
    #precio_descuento=[]
    count_product=0
    product_list=[]
    for product_description in products:

        #contador
        count_product=count_product+1
        #this is the count fraction to check the scrapped elements
        print(f'{count_product} / {len(products)} Elemento escrapeado')

        current_product={'market':'Coto','category': categorias,
                         'currency':'ARG'}
        time.sleep(1)

        #link product
        link_product=product_description.find('a')['href']
        #url_product.append(link_product)
        current_product['link_pagina'] = link_product
        #description product
        description= product_description.find('div',class_='descrip_full').text
        #description_product.append(description)
        current_product['descripcion'] = description

        #precio por unidad
        try:
            unidad = product_description.find('div',class_='price_regular_container').text
            #precio_contado.append(unidad)
            current_product['precio'] = unidad
        except AttributeError:
            unidad = product_description.find('span',class_='atg_store_newPrice').text
            #precio_contado.append(unidad)
            current_product['precio'] = unidad


        #descuento
        try:
            descuento = product_description.find('span',class_='text_price_discount').text
            #promocion.append(descuento)
            current_product['descuento_produccion'] =descuento
        except AttributeError:
            #promocion.append(None)
            current_product['descuento_produccion'] =None

        #precio descuento
        try:
            precio_final=product_description.find('span',class_='price_discount').text
            #precio_descuento.append(precio_final)
            current_product['precio_descuento'] = precio_final
        except AttributeError:
            #precio_descuento.append(None)
            current_product['precio_descuento'] = None

    product_list.append(current_product)

    return product_list
    






In [19]:

def full_urls(partial_link,names_categories):
    #PATH= 'C:\Program Files (x86)\chromedriver.exe'
    PATH= "/usr/lib/chromium-browser/chromedriver"

    link_total = 'https://www.cotodigital3.com.ar'+partial_link

    s = Service(PATH)

    driver = webdriver.Chrome(service=s)
    #driver = webdriver.Chrome("/usr/lib/chromium-browser/chromedriver")
    
    time.sleep(2)
    driver.get(link_total)
    time.sleep(1)
    driver.maximize_window()
    time.sleep(18)
    #next = driver.find_element(By.LINK_TEXT,'Sig')
    #total_urls=[]
    #total_urls.append(link_total)
    
    final_work=[] # list where i hold all the data
    final_work.extend(take_product(names_categories,link_total))


    while True:
            try:
                next = driver.find_element(By.LINK_TEXT,'Sig')
                time.sleep(2)
                next.click()
                pages= driver.current_url
                final_work.extend(take_product(names_categories,pages))
                #total_urls.append(pages)
                time.sleep(7)
            except:
                 break
                 
                 
    sig= driver.find_elements(By.CSS_SELECTOR,'a[title*="Ir a página"]')
    
    for i in range(1,len(sig)):
        try:
            sig= driver.find_elements(By.CSS_SELECTOR,'a[title*="Ir a página"]')
            time.sleep(2)
            sig[i].click()
            pages= driver.current_url
            final_work.extend(take_product(names_categories,pages))
            #total_urls.append(pages)
            time.sleep(7)
        except IndexError:
            sig = driver.find_elements(By.CSS_SELECTOR,'a[title*="Ir a página"]')
            time.sleep(2)
            sig[1].click()
            pages=driver.current_url
            final_work.extend(take_product(names_categories,pages))
            #total_urls.append(pages)
    
    
    #print(f'the total urls: {len(total_urls)}')
    #print(f' the name categories: {names_categories}')
    #nombre=[names_categories]*len(total_urls)
    #df = pd.DataFrame({'categorias':nombre, 'links':total_urls})

    driver.quit()

    return final_work




In [20]:
main_data= [ full_urls(link,title) for link, title in zip(first_step[0],first_step[1])]

1 / 48 Elemento escrapeado
2 / 48 Elemento escrapeado
3 / 48 Elemento escrapeado
4 / 48 Elemento escrapeado
5 / 48 Elemento escrapeado
6 / 48 Elemento escrapeado
7 / 48 Elemento escrapeado
8 / 48 Elemento escrapeado
9 / 48 Elemento escrapeado
10 / 48 Elemento escrapeado
11 / 48 Elemento escrapeado
12 / 48 Elemento escrapeado
13 / 48 Elemento escrapeado
14 / 48 Elemento escrapeado
15 / 48 Elemento escrapeado
16 / 48 Elemento escrapeado
17 / 48 Elemento escrapeado
18 / 48 Elemento escrapeado
19 / 48 Elemento escrapeado
20 / 48 Elemento escrapeado
21 / 48 Elemento escrapeado
22 / 48 Elemento escrapeado
23 / 48 Elemento escrapeado
24 / 48 Elemento escrapeado
25 / 48 Elemento escrapeado
26 / 48 Elemento escrapeado
27 / 48 Elemento escrapeado
28 / 48 Elemento escrapeado
29 / 48 Elemento escrapeado
30 / 48 Elemento escrapeado
31 / 48 Elemento escrapeado
32 / 48 Elemento escrapeado
33 / 48 Elemento escrapeado
34 / 48 Elemento escrapeado
35 / 48 Elemento escrapeado
36 / 48 Elemento escrapeado
3

UnboundLocalError: local variable 'current_product' referenced before assignment

In [10]:
main_data

[(['https://www.cotodigital3.com.ar/sitios/cdigi/browse/catalogo-almac%C3%A9n-golosinas/_/N-1y5dh9i',
   'https://www.cotodigital3.com.ar/sitios/cdigi/browse/catalogo-almac%C3%A9n-golosinas/_/N-1y5dh9i?Nf=product.endDate%7CGTEQ+1.6811712E12%7C%7Cproduct.startDate%7CLTEQ+1.6811712E12&No=48&Nr=AND%28product.sDisp_200%3A1004%2Cproduct.language%3Aespa%C3%B1ol%2COR%28product.siteId%3ACotoDigital%29%29&Nrpp=48',
   'https://www.cotodigital3.com.ar/sitios/cdigi/browse/catalogo-almac%C3%A9n-golosinas/_/N-1y5dh9i?Nf=product.endDate%7CGTEQ+1.6811712E12%7C%7Cproduct.startDate%7CLTEQ+1.6811712E12&No=96&Nr=AND%28product.language%3Aespa%C3%B1ol%2Cproduct.sDisp_200%3A1004%2COR%28product.siteId%3ACotoDigital%29%29&Nrpp=48',
   'https://www.cotodigital3.com.ar/sitios/cdigi/browse/catalogo-almac%C3%A9n-golosinas/_/N-1y5dh9i?Nf=product.endDate%7CGTEQ+1.6811712E12%7C%7Cproduct.startDate%7CLTEQ+1.6811712E12&No=144&Nr=AND%28product.sDisp_200%3A1004%2Cproduct.language%3Aespa%C3%B1ol%2COR%28product.siteId%3AC